In [ ]:
#collecting 

allVideos=[]
freqActivityLeft=[]
freqActivityRight=[]
brain=[]
for count,freq in enumerate(np.unique(flipRates)[:3]):
    index = [i for i,x in enumerate(flipRates) if x == freq]
    startTimesFreq = startTimes[index]
    allVideos = []
    time =  [np.linspace(i+0, i+2, 70) for i in startTimesFreq]
    activity = tToWf(time)
    activity = np.mean(activity, axis=0)
    dwf = [np.diff(i, prepend=i[0]) for i in activity.T]
    dwf = np.array(dwf)

    video = spatial @ dwf
    video = video.reshape(560,560,-1)
    
    # brain.append(np.mean(video,axis=2)) # brain image for all 30 reps, all 100 time points - average activity during a rep
    
    vcLeft = np.mean(video[380:500,120:220,:],axis=(0,1)) # left visual cortex
    freqActivityLeft.append(vcLeft)
    vcRight = np.mean(video[380:500,350:470,:],axis=(0,1)) # right visual cortex
    freqActivityRight.append(vcRight)
    
freqActivityLeft=np.array(freqActivityLeft)
freqActivityRight=np.array(freqActivityRight)
# brain=np.array(brain)

In [ ]:
# fouriers and plotting
fig, axs = plt.subplots(3,2,figsize=(7,7))
t = np.linspace(0,2,70) #i think we define our trial times and thus our sampling rate?
dt = t[1]-t[0] 
N = x.shape[0]
T = N*dt
df = 1/T.max() # our resolution
for i,freq in enumerate(np.unique(flipRates[:3])):
    x = freqActivityLeft[i]
    x = x.reshape(-1)
    
    xmax = x.max()
    max=Sxx[np.where(Sxx==xmax)]
    
    xf = fft(x - x.mean()) #fourier transform
    Sxx = 2 * dt * 2/T * (xf *  xf.conj()) #spectrum
    Sxx = Sxx[:int(len(x)/2)] # our data is real, so ignore negative freqs
    fNQ = 1/dt/2 # nyquist freq - expected highest freq
    faxis = np.arange(0,fNQ,df)

    max_y = Sxx.reshape(-1).max()  # Find the maximum y value
    max_x = faxis[Sxx.argmax()]

    axs[i,0].plot(faxis,Sxx)
    axs[i,0].set_xlabel('frequency')
    axs[i,0].set_ylabel('Power [$\mu V^2$/Hz]')
    freq =np.unique(flipRates)[i]
    axs[i,0].set_title(f'freq {freq} left side')
    axs[i,0].autoscale(axis='x',tight='true')
    # axs[i,0].set_ylim([0,200000])
    axs[i,0].text(s=f'max: {max_x:.2f}',x=10,y=10)

    x = freqActivityRight[i]
    x = x.reshape(-1)
    xmax = x.max()
    xf = fft(x - x.mean()) 
    Sxx = 2 * dt * 2/T * (xf *  xf.conj()) 
    Sxx = Sxx[:int(len(x)/2)]
    fNQ = 1/dt/2
    faxis = np.arange(0,fNQ,df)

    max_y = Sxx.reshape(-1).max()  # Find the maximum y value
    max_x = faxis[Sxx.argmax()]

    axs[i,1].plot(faxis,Sxx)
    # axs[i,1].set_xlabel('frequency')
    # axs[i,1].set_ylabel('Power [$\mu V^2$/Hz]')
    axs[i,1].set_title(f'freq {freq} right side')
    axs[i,1].autoscale(axis='x',tight='true')
    # axs[i,1].set_ylim([0,200000])
    axs[i,1].text(s=f'max: {max_x:.2f}',x=10,y=10)

    fig.tight_layout()
    